In [1]:
import pandas as pd
sales_data=pd.read_excel('Pizza_Sale.xlsx')
ingredients_df=pd.read_excel('Pizza_ingredients.xlsx')

In [3]:
sales_data.dropna(subset=['pizza_name_id','total_price','pizza_category','pizza_ingredients','pizza_name'],inplace=True)
ingredients_df.dropna(subset=['Items_Qty_In_Grams'],inplace=True)
sales_data['order_time']=pd.to_datetime(sales_data['order_time'],format='%H:%M:%S')

In [6]:
ingredients_df

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...,...
513,veggie_veg_s,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,veggie_veg_s,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,veggie_veg_s,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,veggie_veg_s,The Vegetables + Vegetables Pizza,Spinach,15.0


In [30]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


# Group the sales data by pizza_name_id
pizza_sales_group = sales_data.groupby('pizza_name_id')

# Dictionary to hold the forecast for each pizza
forecasts = {}

# Loop for each pizza type to fit the ARIMA model and forecast sales
for pizza_id, group in pizza_sales_group:
    # Set date as index and resample to daily frequency
    group = group.set_index('order_date')
    group = group['quantity'].resample('D').sum().fillna(0)
    
    # Fit ARIMA model (you can adjust the order based on better performance)
    model = ARIMA(group, order=(1, 1, 1))
    model_fit = model.fit()
    
    # Forecast the next 7 days
    forecast = model_fit.forecast(steps=7)
    
    # Generate future dates
    last_date = group.index[-1]
    future_dates = pd.date_range(last_date, periods=7, freq='D')
    
    # Store the forecast
    forecasts[pizza_id] = pd.Series(forecast, index=future_dates)
    
    
ingredients_df['Items_Qty_In_Grams'] = pd.to_numeric(ingredients_df['Items_Qty_In_Grams'], errors='coerce')
ingredients_df['Items_Qty_In_Grams'].fillna(0, inplace=True)

for pizza_name_id, forecast in forecasts.items():
    print(f"Pizza ID: {pizza_name_id}, Forecast: {forecast}")

# Create a dictionary to store the total ingredient prediction
total_ingredients_forecast = {}

# Proceed if forecasts are available
if forecasts:
    # Loop over each pizza type and its forecast
    for pizza_name_id, forecast in forecasts.items():
        # Filter the ingredients for the current pizza type
        pizza_ingredients = ingredients_df[ingredients_df['pizza_name_id'] == pizza_name_id]

        # Ensure there are ingredients for this pizza
        if pizza_ingredients.empty:
            print(f"No ingredients found for pizza ID {pizza_name_id}")
            continue

        # Loop through each day in the forecast
        for date, sales_prediction in forecast.items():
            # Handle NaN forecast values by replacing them with 0
            sales_prediction = 0 if pd.isna(sales_prediction) else sales_prediction

            # Loop through each ingredient for the current pizza
            for _, row in pizza_ingredients.iterrows():
                ingredient = row['pizza_ingredients']
                quantity_per_pizza = row['Items_Qty_In_Grams']

                # Handle NaN in quantity_per_pizza by treating them as 0
                quantity_per_pizza = 0 if pd.isna(quantity_per_pizza) else quantity_per_pizza

                # Calculate the total quantity of the ingredient needed for the predicted sales
                total_quantity = sales_prediction * quantity_per_pizza

                # Add this to the total ingredients forecast
                if ingredient not in total_ingredients_forecast:
                    total_ingredients_forecast[ingredient] = 0
                total_ingredients_forecast[ingredient] += total_quantity

# Display the total ingredients needed for the next week
if total_ingredients_forecast:
    print("\nTotal ingredients required for the next 7 days:")
    for ingredient, total_quantity in total_ingredients_forecast.items():
        print(f"{ingredient}: {total_quantity:.2f} grams")
else:
    print("No ingredients forecast available.")






Pizza ID: bbq_ckn_l, Forecast: 2015-12-31         NaN
2016-01-01    2.674990
2016-01-02    2.559582
2016-01-03    2.553853
2016-01-04    2.553569
2016-01-05    2.553555
2016-01-06    2.553554
Freq: D, Name: predicted_mean, dtype: float64
Pizza ID: bbq_ckn_m, Forecast: 2015-12-31         NaN
2016-01-01    2.770922
2016-01-02    2.634433
2016-01-03    2.619276
2016-01-04    2.617593
2016-01-05    2.617406
2016-01-06    2.617385
Freq: D, Name: predicted_mean, dtype: float64
Pizza ID: bbq_ckn_s, Forecast: 2015-12-31         NaN
2016-01-01    1.332054
2016-01-02    1.322919
2016-01-03    1.323170
2016-01-04    1.323163
2016-01-05    1.323163
2016-01-06    1.323163
Freq: D, Name: predicted_mean, dtype: float64
Pizza ID: big_meat_s, Forecast: 2015-12-31         NaN
2016-01-01    5.194151
2016-01-02    5.240550
2016-01-03    5.237879
2016-01-04    5.238032
2016-01-05    5.238024
2016-01-06    5.238024
Freq: D, Name: predicted_mean, dtype: float64
Pizza ID: brie_carre_s, Forecast: 2015-12-31   

C:\Users\Katana\AppData\Local\Temp\ipykernel_38028\3840184751.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ingredients_df['Items_Qty_In_Grams'].fillna(0, inplace=True)
